In [35]:
import sqlite3 
from sqlite3 import Error 
import pandas as pd

### Объявляем вспомогательные функции

In [36]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [37]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

## Шаг 1
### Устанавливаем соединение с базой, создаем таблицы и загружаем csv-файлы в таблицы

In [38]:
connection = create_connection("transaction.db")

Connection to SQLite DB successful


In [39]:
create_transation_table = """
    CREATE TABLE IF NOT EXISTS transaction_bd (
        TRANSACTION_ID INTEGER,
        TX_DATETIME NUMERIC,
        CUSTOMER_ID INTEGER,
        TERMINAL_ID INTEGER,
        TX_AMOUNT REAL DEFAULT 0
    );
"""

In [40]:
execute_query(connection, create_transation_table)  

Query executed successfully


In [41]:
df = pd.read_csv("transactions_for_dz2.csv")
df.to_sql("transaction_bd", connection, if_exists="append", index=False)

1048575

In [42]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [43]:
pd.read_sql(f"SELECT * FROM transaction_bd", connection)

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT
0,0,2023-01-01 00:00:31,596,3156,533.07
1,1,2023-01-01 00:02:10,4961,3412,808.56
2,2,2023-01-01 00:07:56,2,1365,1442.94
3,3,2023-01-01 00:09:29,4128,8737,620.65
4,4,2023-01-01 00:10:34,927,9906,490.66
...,...,...,...,...,...
1048570,1048570,2023-04-20 10:07:13,2380,3780,325.64
1048571,1048571,2023-04-20 10:07:28,738,5151,20.38
1048572,1048572,2023-04-20 10:07:33,1000,5417,182.79
1048573,1048573,2023-04-20 10:07:39,3028,6439,455.44


In [45]:
create_customer_table = """
    CREATE TABLE IF NOT EXISTS customer_bd (
        CLIENT_ID INTEGER,
        START_DT NUMERIC,
        END_DT NUMERIC,
        CLIENT_NAME TEXT,
        YEAR_BIRTH TEXT
    );
"""

In [46]:
execute_query(connection, create_customer_table)  

Query executed successfully


In [47]:
df = pd.read_csv("client_info.csv", sep=";")
df.to_sql("customer_bd", connection, if_exists="append", index=False)

4988

In [48]:
pd.read_sql(f"SELECT * FROM customer_bd", connection)

,CLIENT_ID,START_DT,END_DT,CLIENT_NAME,YEAR_BIRTH
0,2213,2015-07-16,2021-01-01,Olivia,1979
1,1148,2015-07-17,2021-01-01,Emma,1979
2,2293,2015-07-18,2021-01-01,Charlotte,1979
3,1867,2015-07-19,2021-01-01,Amelia,1979
4,1767,2015-07-20,2021-01-01,Ava,1979
...,...,...,...,...,...
4983,4737,2023-04-20,2999-12-31,Havilah,2000
4984,2301,2023-04-20,2999-12-31,Hazelyn,2000
4985,3238,2023-04-20,2999-12-31,Helaina,2000
4986,3946,2023-04-20,2999-12-31,Helene,2000


## Шаг 2

### A. Вывести список всех клиентов, у которых между двумя ближайшими транзакциями был перерыв больше 35 дней хотя бы один раз.

In [49]:
df = pd.read_sql(
    f"""
        SELECT
            s.CLIENT_ID,
            s.CLIENT_NAME,
            MAX(s.diff) AS MAX_DIFF
        FROM
        (
            SELECT
                c.CLIENT_ID,
                c.CLIENT_NAME,
                JULIANDAY(t.TX_DATETIME) - JULIANDAY(LAG(t.TX_DATETIME, 1) OVER (
                    PARTITION BY t.CUSTOMER_ID 
                    ORDER BY t.TX_DATETIME
                )) AS DIFF
            FROM
                customer_bd c
            LEFT JOIN
                transaction_bd t
                    ON t.CUSTOMER_ID = c.CLIENT_ID
            WHERE
                DATE(c.END_DT) >= DATE('2023-05-01')
        ) s
        WHERE
            s.DIFF > 35
        GROUP BY
            s.CLIENT_ID
        ORDER BY 
            s.CLIENT_ID
        ;
    """, connection)
df

,CLIENT_ID,CLIENT_NAME,MAX_DIFF
0,24,Kiarra,40.15
1,480,Vaishnavi,54.37
2,639,Malayna,35.32
3,707,Becca,35.12
4,812,Avionna,83.00
5,896,Macarena,50.48
6,1129,Kavya,41.84
7,1299,Nalah,65.27
8,1334,Blakley,55.79
9,1459,Nalia,44.90


### B. Вывести список клиентов, у которых максимальная сумма транзакции (имеется ввиду максимальное значение TX_AMOUNT) больше 50000. В ответе должен быть id клиента, максимальное значение суммы транзакций, минимальное значение суммы транзакций.

In [50]:
df = pd.read_sql(
    f"""
        SELECT
            s.CLIENT_ID,
            s.CLIENT_NAME,
            s.MAX_TX_AMOUNT,
            s.MIN_TX_AMOUNT
        FROM
        (
            SELECT
                c.CLIENT_ID,
                c.CLIENT_NAME,
                MAX(t.TX_AMOUNT) AS MAX_TX_AMOUNT,
                MIN(t.TX_AMOUNT) AS MIN_TX_AMOUNT
            FROM
                customer_bd c
            LEFT JOIN
                transaction_bd t
                    ON t.CUSTOMER_ID = c.CLIENT_ID
            WHERE
                DATE(c.END_DT) >= DATE('2023-05-01')
            GROUP BY
                c.CLIENT_ID
        ) s
        WHERE
            s.MAX_TX_AMOUNT > 50000
        ORDER BY 
            s.CLIENT_ID
        ;
    """, connection)
df

,CLIENT_ID,CLIENT_NAME,MAX_TX_AMOUNT,MIN_TX_AMOUNT
0,3013,Arissa,77212.50,1.46
1,3494,Atlee,53213.00,85.76
2,4253,Zeina,51937.25,37.95


### C. Вывести топ-10 клиентов (сортируя их по убыванию id), у которых сумма дневных транзакций больше суммы вечерних транзакций. Условимся, что дневными считаются транзакции, которые были произведены до 12 часов, а вечерними - после 12 часов включительно. 

Формулировка "Топ-10 клиентов (сортируя их по убыванию id)" звучит немного расплывчато для меня, поэтому я сделал два варианта.

Первый вариант, топ-10 с сортировкой по CLIENT_ID по убыванию без учета суммы:

In [51]:
df = pd.read_sql(
    f"""
        SELECT
            s.CLIENT_ID,
            s.CLIENT_NAME,
            s.DAY_TX_AMOUNT,
            s.AFTERNOON_TX_AMOUNT
        FROM
        (
            SELECT
                c.CLIENT_ID,
                c.CLIENT_NAME,
                SUM(CASE WHEN CAST(strftime('%H', t.TX_DATETIME) AS INTEGER) <  12 THEN t.TX_AMOUNT ELSE 0 END) AS DAY_TX_AMOUNT,
                SUM(CASE WHEN CAST(strftime('%H', t.TX_DATETIME) AS INTEGER) >= 12 THEN t.TX_AMOUNT ELSE 0 END) AS AFTERNOON_TX_AMOUNT
            FROM
                customer_bd c
            LEFT JOIN
                transaction_bd t
                    ON t.CUSTOMER_ID = c.CLIENT_ID
            WHERE
                DATE(c.END_DT) >= DATE('2023-05-01')
            GROUP BY
                c.CLIENT_ID
        ) s
        WHERE
            s.DAY_TX_AMOUNT > s.AFTERNOON_TX_AMOUNT
        ORDER BY
            s.CLIENT_ID DESC
        LIMIT 10
        ;
    """, connection)
df

,CLIENT_ID,CLIENT_NAME,DAY_TX_AMOUNT,AFTERNOON_TX_AMOUNT
0,4997,Fatou,68153.52,63135.10
1,4993,Oakland,43648.09,41244.55
2,4990,Nalayah,86921.94,86536.14
3,4987,Kaylah,12239.39,11547.78
4,4983,Raylynn,109737.94,97444.22
5,4982,Sarayu,45214.41,40213.19
6,4978,Rayleigh,3210.61,3001.03
7,4976,Brayleigh,71894.97,61467.88
8,4973,Hollis,65297.68,64986.58
9,4972,Manha,3092.83,2702.23


Второй вариант, топ-10 по суммам дневных транзакций + сортировка по CLIENT_ID по убыванию:

In [52]:
df = pd.read_sql(
    f"""
        SELECT
            *
        FROM
        (
            SELECT
                s.CLIENT_ID,
                s.CLIENT_NAME,
                s.DAY_TX_AMOUNT,
                s.AFTERNOON_TX_AMOUNT
            FROM
            (
                SELECT
                    c.CLIENT_ID,
                    c.CLIENT_NAME,
                    SUM(CASE WHEN CAST(strftime('%H', t.TX_DATETIME) AS INTEGER) <  12 THEN t.TX_AMOUNT ELSE 0 END) AS DAY_TX_AMOUNT,
                    SUM(CASE WHEN CAST(strftime('%H', t.TX_DATETIME) AS INTEGER) >= 12 THEN t.TX_AMOUNT ELSE 0 END) AS AFTERNOON_TX_AMOUNT
                FROM
                    customer_bd c
                LEFT JOIN
                    transaction_bd t
                        ON t.CUSTOMER_ID = c.CLIENT_ID
                WHERE
                    DATE(c.END_DT) >= DATE('2023-05-01')
                GROUP BY
                    c.CLIENT_ID
            ) s
            WHERE
                s.DAY_TX_AMOUNT > s.AFTERNOON_TX_AMOUNT
            ORDER BY
                s.DAY_TX_AMOUNT DESC
            LIMIT 10
        )
        ORDER BY
            CLIENT_ID DESC
        ;
    """, connection)
df

,CLIENT_ID,CLIENT_NAME,DAY_TX_AMOUNT,AFTERNOON_TX_AMOUNT
0,4253,Zeina,396603.39,152940.29
1,4163,Maddisyn,436430.76,328722.87
2,3494,Atlee,359443.31,188432.45
3,3406,Xylah,293047.48,221436.26
4,3343,Sunnie,364307.13,213814.95
5,2781,Jailyn,293819.57,205923.23
6,2110,Soliha,320822.11,247444.42
7,2080,Maggie,280513.74,222672.13
8,1918,Jovi,339043.39,191679.21
9,1883,Kayci,427525.21,199022.80


### D. Найти для каждого клиента день, когда у него была максимальная сумма транзакции, то есть вывести клиента, дату транзакции, сумму транзакции.

Примечание: я решил также выводить клиентов без транзакций, потому что в условии задачи мы отталкиваемся от таблицы клиентов.

In [53]:
df = pd.read_sql(
    f"""
        WITH gt AS
        (SELECT
             CUSTOMER_ID,
             MAX(TX_AMOUNT) AS MAX_TX_AMOUNT
         FROM
             transaction_bd
         GROUP BY
             CUSTOMER_ID
        )
        SELECT DISTINCT
            c.CLIENT_ID,
            c.CLIENT_NAME,
            t.TX_DATETIME,
            t.TX_AMOUNT
        FROM
            customer_bd c
        LEFT JOIN
            transaction_bd t
                ON t.CUSTOMER_ID = c.CLIENT_ID
        LEFT JOIN
            gt ON t.CUSTOMER_ID = gt.CUSTOMER_ID
        WHERE
                (t.TX_AMOUNT = gt.MAX_TX_AMOUNT OR gt.MAX_TX_AMOUNT IS NULL)
            AND DATE(c.END_DT) >= DATE('2023-05-01')
        ORDER BY
            c.CLIENT_ID
        ;
    """, connection)
df

,CLIENT_ID,CLIENT_NAME,TX_DATETIME,TX_AMOUNT
0,0,Eleni,2023-02-01 05:30:38,1258.86
1,1,Jocelyn,2023-02-07 18:42:49,17152.00
2,2,Bria,2023-02-21 20:09:34,1720.11
3,5,Zimal,2023-03-21 13:00:37,1156.11
4,6,Raeya,2023-03-20 13:13:07,369.26
...,...,...,...,...
3733,4993,Oakland,2023-01-13 15:50:46,1054.57
3734,4996,Royal,2023-01-11 09:47:27,224.62
3735,4997,Fatou,2023-01-17 21:33:44,1385.73
3736,11111,Oksi,None,NaN


### E. Вычислить интегральное распределение суммы транзакций для каждого дня за весь период, то есть вывести дату транзакции, сумму транзакции и интегральное распределение по сумме.

In [54]:
df = pd.read_sql(
    f"""
        SELECT
            s.TX_DATE,
            s.SUM_TX_AMOUNT,
            CUME_DIST() OVER (ORDER BY s.SUM_TX_AMOUNT) AS CUME_DIST
        FROM
        (
            SELECT
                DATE(t.TX_DATETIME) AS TX_DATE,
                SUM(t.TX_AMOUNT) AS SUM_TX_AMOUNT
            FROM
                transaction_bd t
            GROUP BY
                DATE(t.TX_DATETIME)
        ) s
        ORDER BY
            s.TX_DATE
        ;
    """, connection)
df

,TX_DATE,SUM_TX_AMOUNT,CUME_DIST
0,2023-01-01,4827656.26,0.02
1,2023-01-02,4862551.41,0.03
2,2023-01-03,5058973.71,0.21
3,2023-01-04,4938142.47,0.04
4,2023-01-05,5002954.23,0.11
...,...,...,...
105,2023-04-16,5299386.81,0.91
106,2023-04-17,5100973.31,0.32
107,2023-04-18,5233557.39,0.75
108,2023-04-19,5194846.44,0.64


### Закрываем соединение

In [55]:
connection.close()